# Các công việc triển khai

Trong file này sẽ thực hiện các việc.

- Cho data ảnh từ các tập train, valid, test qua module 1 và save lại kết quả vào 3 file csv tương ứng.
- Cụ thể với mỗi một ảnh, ảnh gốc sẽ resize về 1024x768, cắt lấy 12 miếng theo lưới 256x256. Như vậy tổng cộng thu được 13 miếng (tất cả resize về 224x224). Cuối cùng 13 miếng này cho qua module 1 thu được 13 vector 3 chiều (tương ứng cho xác suất thuộc về 3 nhãn của 1 miếng ảnh). Như vậy 1 bản ghi sẽ được lưu trữ dưới dạng 13 chiều 3 chiều này (tức 39 chiều).
- Sau khi đã save 3 file csv cho mỗi tập. Thực hiện phân tích phân phối dữ liệu các trường thu được (trước hết chỉ thực hiện với 3 chiều đầu tiên - tức chỉ cho miếng toàn cục nhất). Sau đó mới phân tích đến cả 39 chiều cùng lúc.
- Các thông tin đem phân tích có thể như phân phối của chiều dữ liệu này có phải phân phối chuẩn hay không. Hệ số tương quan của các chiều này với nhau (tại vì chúng có thể có hiện tượng đa cộng tuyến).
- Sau khi xác định được phân phối của dữ liệu thì đề xuất các phương pháp để tạo sinh thêm dữ liệu mới dựa trên dữ liệu phân phối đã có.

In [ ]:
# Khởi tạo model của module 1 để giúp dự đoán nhãn của 1 ảnh
!pwd
# import os
# os.chdir("../../src/models/module1")
# !pwd

# import torch
# from efficient_net import H0_EfficientNetB0

# model = H0_EfficientNetB0()
# ''' 
# Model là 1 mạng CNN EfficientNetB0 thông thường với đầu ra phân loại 3 nhãn
# ''' 
# model_path = '../../weights/1725034685_phase2_280824_module1_H0_EfficientNetB0/best_loss.pth'

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming H0_EfficientNetB0 is imported correctly
os.chdir("../../src/models/module1")
from efficient_net import H0_EfficientNetB0

def load_model(model_path):
    model = H0_EfficientNetB0()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
    model.eval()
    return model, device

def process_image(image_path, model, device):
    # Resize to 1024x768 and create 12 256x256 patches + 1 full image
    image = Image.open(image_path).convert('RGB')
    image = image.resize((1024, 768))
    patches = []
    patches.append(image)
    for i in range(3):
        for j in range(4):
            patch = image.crop((j*256, i*256, (j+1)*256, (i+1)*256))
            patches.append(patch)
    
    # Resize all to 224x224 and process
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # results = []
    # for patch in patches:
    #     input_tensor = transform(patch).unsqueeze(0).to(device)
    #     with torch.no_grad():
    #         output = model(input_tensor)
    #     probabilities = output
    #     # probabilities = torch.nn.functional.softmax(output, dim=1)
    #     results.extend(probabilities.cpu().numpy().flatten())
    
    results = []
    # Convert all patches to tensors and stack them into a single batch
    batch_tensor = torch.stack([transform(patch) for patch in patches]).to(device)
    # Run the model inference on the entire batch
    with torch.no_grad():
        outputs = model(batch_tensor)
    # Since each output corresponds to one patch, you can directly extract probabilities
    probabilities = outputs
    # If you want softmax probabilities, you can uncomment the following line:
    # probabilities = torch.nn.functional.softmax(outputs, dim=1)
    # Flatten the results and extend to the results list
    results.extend(probabilities.cpu().numpy().flatten())

    return results

def process_dataset(data_dir, model, device, output_file):
    results = []
    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(root, file)
                result = process_image(image_path, model, device)
                results.append([image_path] + result)
    
    df = pd.DataFrame(results, columns=['image_path'] + [f'dim_{i}' for i in range(39)])
    df.to_csv(output_file, index=False)
    return df

def analyze_distribution(df):
    # Analyze first 3 dimensions (global patch)
    for i in range(3):
        col = f'dim_{i}'
        _, p_value = stats.normaltest(df[col])
        print(f"Normality test for {col}: p-value = {p_value}")
        
        plt.figure(figsize=(10, 5))
        sns.histplot(df[col], kde=True)
        plt.title(f"Distribution of {col}")
        plt.show()
    
    # Correlation matrix for all 39 dimensions
    corr_matrix = df.iloc[:, 1:].corr()
    plt.figure(figsize=(20, 20))
    sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
    plt.title("Correlation Matrix of All Dimensions")
    plt.show()

def main():
    model_path = '../../weights/1725034685_phase2_280824_module1_H0_EfficientNetB0/best_loss.pth'
    model, device = load_model(model_path)
    
    data_dirs = {
        'train': '../../data/train',
        'valid': '../../data/valid',
        'test': '../../data/test'
    }
    
    for split, data_dir in data_dirs.items():
        output_file = f'../../data/processed/{split}_features.csv'
        df = process_dataset(data_dir, model, device, output_file)
        print(f"Processed {split} dataset. Shape: {df.shape}")
        
        analyze_distribution(df)
    
    print("Processing and analysis complete.")

if __name__ == "__main__":
    main()
